<a href="https://colab.research.google.com/github/fred-ykv/Value-Investing-In-Python/blob/master/Revis%C3%A3o_VALUATION_FINVIZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import seaborn as sns
from urllib.request import Request, urlopen

pd.set_option('display.max_colwidth', 25)

In [3]:
!pip install yfinance --upgrade --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 57.2 MB/s eta 0:00:00


In [4]:
!pip install yfinance

import yfinance as yf
yf.pdr_override()
import plotly.graph_objs as go
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install git+https://github.com/quantopian/pyfolio
import pyfolio as pf
import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/quantopian/pyfolio to /tmp/pip-req-build-0s09ln2q
  Running command git clone --filter=blob:none --quiet https://github.com/quantopian/pyfolio /tmp/pip-req-build-0s09ln2q
  Resolved https://github.com/quantopian/pyfolio to commit 4b901f6d73aa02ceb6d04b7d83502e5c6f2e81aa
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.3 MB/s eta 0:00:00
  Created wheel for pyfolio: filename=pyfolio-0.9.2+75.g4b901f6-py3-none-any.whl size=75775 sha256=c16064cfee8e89c3b3f494769950636a689620c259f0a0ffff952dc61f6d6249
  Stored in directory: /tmp/pip-ephem-wheel-cache-8tvf7ti8/wheels/b2/62/f1/189c72d02d1cd5e075104a3c282f3e442e03386366b50b5855
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.w

/usr/local/lib/python3.9/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


# Entrada do Ticker

In [6]:
symbol = input('Enter a ticker: ')
print ('Getting data for ' + symbol + '...\n')

Enter a ticker: wire
Getting data for wire...



# Set up scraper

In [7]:
url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

In [8]:
def get_fundamentals():
    try:
        # Find fundamentals table
        fundamentals = pd.read_html(str(html), attrs = {'class': 'snapshot-table2'})[0]
        
        # Clean up fundamentals dataframe
        fundamentals.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
        colOne = []
        colLength = len(fundamentals)
        for k in np.arange(0, colLength, 2):
            colOne.append(fundamentals[f'{k}'])
        attrs = pd.concat(colOne, ignore_index=True)
    
        colTwo = []
        colLength = len(fundamentals)
        for k in np.arange(1, colLength, 2):
            colTwo.append(fundamentals[f'{k}'])
        vals = pd.concat(colTwo, ignore_index=True)
        
        fundamentals = pd.DataFrame()
        fundamentals['Attributes'] = attrs
        fundamentals['Values'] = vals
        fundamentals = fundamentals.set_index('Attributes')
        return fundamentals

    except Exception as e:
        return e

In [9]:
def get_news():
    try:
        # Find news table
        news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]
        links = []
        for a in html.find_all('a', class_="tab-link-news"):
            links.append(a['href'])
        
        # Clean up news dataframe
        news.columns = ['Date', 'News Headline']
        news['Article Link'] = links
        news = news.set_index('Date')
        return news

    except Exception as e:
        return e

In [10]:
def get_insider():
    try:
        # Find insider table
        insider = pd.read_html(str(html), attrs = {'class': 'body-table'})[0]
        
        # Clean up insider dataframe
        insider = insider.iloc[1:]
        insider.columns = ['Trader', 'Relationship', 'Date', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']
        insider = insider[['Date', 'Trader', 'Relationship', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']]
        insider = insider.set_index('Date')
        return insider

    except Exception as e:
        return e

In [11]:
print ('\nFundamental Ratios: ')
print(get_fundamentals())

print ('\nRecent News: ')
print(get_news())

print ('\nRecent Insider Trades: ')
print(get_insider())


Fundamental Ratios: 
                 Values
Attributes             
Index                 -
Market Cap        3.31B
Income          717.80M
Sales             3.02B
Book/sh           99.60
...                 ...
ATR                7.25
Volatility  5.18% 4.25%
Prev Close       175.65
Price            173.76
Change           -1.08%

[72 rows x 1 columns]

Recent News: 
                              News Headline              Article Link
Date                                                                 
Mar-16-23 05:05PM  Encore Wire Unveils E...  https://finance.yahoo...
Mar-13-23 10:25AM  Zacks.com featured hi...  https://finance.yahoo...
10:12AM            Zacks.com featured hi...  https://finance.yahoo...
Mar-10-23 07:19AM  4 Stocks With Solid N...  https://finance.yahoo...
06:38AM            4 Stocks With Increas...  https://finance.yahoo...
...                                     ...                       ...
Jul-15-22 12:00PM  Encore Wire Announces...  https://finance.yahoo..

In [12]:
print ('Fundamental Ratios: ')
fundamentals = get_fundamentals()
if 'Market Cap' in fundamentals.index:
    print(fundamentals.loc['Market Cap'])
else:
    print('Market Cap is not listed.')

Fundamental Ratios: 
Values    3.31B
Name: Market Cap, dtype: object


Valor de Mercado

In [13]:
market_cap_str = fundamentals.loc['Market Cap']['Values']
market_cap_str = market_cap_str.strip()

if market_cap_str[-1] == 'B':
    market_cap = float(market_cap_str[:-1]) * 1000000000
elif market_cap_str[-1] == 'M':
    market_cap = float(market_cap_str[:-1]) * 1000000
else:
    market_cap = float(market_cap_str)

print(f'Market Cap: ${market_cap:,.2f}')

Market Cap: $3,310,000,000.00


ROI

In [14]:
roi_str = fundamentals.loc['ROI']['Values']
roi_str = roi_str.replace('%', '')
roi = float(roi_str)
roi_perc = roi/100
roi_perc_str = str(roi_perc).replace('.', ',')

print(f'ROI: {roi_perc_str}')

ROI: 0,389


ROE

In [15]:
roe_str = fundamentals.loc['ROE']['Values']
roe_str = roe_str.replace('%', '')
roe = float(roe_str)
roe_perc = roe/100
roe_perc_str = str(roe_perc).replace('.', ',')

print(f'ROE: {roe_perc_str}')

ROE: 0,44


Payout

In [16]:
payout_str = fundamentals.loc['Payout']['Values']
payout_str = payout_str.replace('%', '')
payout = float(payout_str)
payout_perc = payout/100
payout_perc_str = str(payout_perc).replace('.', ',')

print(f'Payout: {payout_perc_str}')

Payout: 0,002


Dividend Yield

In [17]:
div_str = fundamentals.loc['Dividend %']['Values']
div_str = div_str.replace('%', '')
div = float(div_str)
div_perc = div/100
div_perc_str = str(div_perc).replace('.', ',')

print(f'Dividend %: {div_perc_str}')

Dividend %: 0,0005


Crescimento dos Lucros nos próximos 5 anos

In [18]:
epsn5_str = fundamentals.loc['EPS next 5Y']['Values']
epsn5_str = epsn5_str.replace('%', '')
epsn5 = float(epsn5_str)
epsn5_perc = epsn5/100
epsn5_perc_str = str(epsn5_perc).replace('.', ',')

print(f'EPS next 5: {epsn5_perc_str}')

EPS next 5: 0,1


LPA - Lucro por Ação

In [19]:
eps_str = fundamentals.loc['EPS (ttm)']['Values']
eps = float(eps_str)
eps_str = str(eps).replace('.', ',')

print(f'EPS: {eps_str}')

EPS: 36,91


VPA - Valor Patrimonial por Ação

In [20]:
VPA_str = fundamentals.loc['Book/sh']['Values']
VPA = float(VPA_str)
VPA_str = str(VPA).replace('.', ',')

print(f'VPA: {VPA}')


VPA: 99.6


Beta

In [21]:
beta_str = fundamentals.loc['Beta']['Values']
beta = float(beta_str)
beta_str = str(beta).replace('.', ',')

print(f'Beta: {beta_str}')

Beta: 1,35


Taxa de Reinvestimento do Lucro Líquido (bLL)

In [22]:
tax_reinvest_ll = 1 - payout_perc
tax_reinvest_ll_str = str(tax_reinvest_ll).replace('.', ',')

print(f'Taxa de Reinvestimento bLL: {tax_reinvest_ll:.3f}')

Taxa de Reinvestimento bLL: 0.998


gLL - Crescimento do Lucro Líquido

In [23]:
roe_str = fundamentals.loc['ROE']['Values']
roe_str = roe_str.replace('%', '')
roe = float(roe_str)
roe_perc = roe/100

gll = tax_reinvest_ll*roe_perc
gll_str = str(gll).replace('.', ',')

print(f'gLL: {gll:.3f}')

gLL: 0.439


Ganho de Alavancagem

In [24]:
gain_leverage = roe_perc - roi_perc
gain_leverage_str = str(gain_leverage).replace('.', ',')

print(f'Ganho de Alavancagem: {gain_leverage:.3f}')

Ganho de Alavancagem: 0.051


Grau de Alavancagem Financeira

In [25]:
gaf = roe_perc/roi_perc
gaf_str = str(gaf).replace('.', ',')

print(f'Grau de Alavancagem Financeira: {gaf:.3f}')

Grau de Alavancagem Financeira: 1.131


Taxa de Juros Livre de Risco (Rf) - Treasury Bond 10Y

In [26]:
# Requisição Url
url = "https://ycharts.com/indicators/10_year_treasury_rate"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Parser
html = soup(webpage, "html.parser")

# Buscando Valor
xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    Rf = float(value.replace(',', '.'))
    Rf_str = str(Rf).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {Rf_str}')

Taxa de Juros Livre de Risco (Rf): 3,56%


In [27]:
url = 'https://ycharts.com/indicators/us_10year_government_bond_interest_rate'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    interest_rate = float(value.replace(',', '.'))
    interest_rate_str = str(interest_rate).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {interest_rate_str}')

Taxa de Juros Livre de Risco (Rf): 3,75%


Taxa de Inflação Americana

In [28]:
# Requisição Url
url = "https://ycharts.com/indicators/us_consumer_price_index_yoy"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Parser
html = soup(webpage, "html.parser")

# Buscando Valor
xpath = '/html/body/main/div/div[2]/div/div/div[2]'
value_elem = html.find('div', {'class':'key-stat-title'}).get_text()

if value_elem is not None:
    value = value_elem.strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    inflation = float(value.replace(',', '.'))
    inflation_str = str(inflation).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Inflação Americana:: {inflation_str}')

Taxa de Inflação Americana:: 6,04%


S&P 500 - Retorno 10Y

In [29]:
def calcula_retorno(dataframe):
  df = dataframe.copy()  
  df['daily_return']        = df.iloc[:,0].pct_change()
  df['acum_return']         = (1 +  df['daily_return']).cumprod()
  df['acum_return'].iloc[0] = 1
  df.rename(columns={df.columns[0]: "price" }, inplace=True)
  return df

sp500 = yf.download('^GSPC')[['Adj Close']]
sp500

sp500 = calcula_retorno(sp500)
sp500

Ten_year_return = sp500['acum_return'].iloc[-1] * 100 - 100
print(f'Retorno acumulado 10Y: {Ten_year_return:.1f}%')

mean_yearly_return = sp500['daily_return'].mean() * 100 * 365
print(f'Retorno médio anual 10Y: {mean_yearly_return:.2f}%')


[*********************100%***********************]  1 of 1 completed
Retorno acumulado 10Y: 22097.4%
Retorno médio anual 10Y: 10.88%


In [30]:
url = 'https://br.investing.com/economic-calendar/brazilian-cpi-410'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

xpath = '/html/body/div[5]/section/div[6]/span[2]/div'
value_elem = html.find('div', {'class':'inner_cell'})

if value_elem is not None:
    value = value_elem.get_text().strip()

    # Removendo Caracteres
    interest_rate = float(value.replace(',', '.'))
    interest_rate_str = str(interest_rate).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {interest_rate_str}')

Taxa de Inflação Brasil

In [31]:
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen

url = 'https://www.bcb.gov.br/'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

value_elem = html.select_one('div.valor')

if value_elem is not None:
    value = value_elem.get_text().strip()
    value = value.split(' ')[0] # Removendo Strings
    value = value.replace('%', '')

    # Removendo Caracteres
    interest_rate = float(value.replace(',', '.'))
    interest_rate_str = str(interest_rate).replace('.', ',') + '%' # Corrigindo Erro

    print(f'Taxa de Juros Livre de Risco (Rf): {interest_rate_str}')

# **Balanço Patrimonial**

In [32]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/balance-sheet'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

Patrimônio Líquido (PL)

In [33]:
# Encontrando o valor de Total Shareholder's Equity
total_shareholders_equity = soup.find('td', text="Total Shareholder's Equity")
value = total_shareholders_equity.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_12 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
value_formatted = '${:,.2f}'.format(value_in_millions_12)

# Imprimindo o valor formatado
print(value_formatted)


AttributeError: ignored

Empréstimos e Financiamentos

In [ ]:
# Encontrando o valor de Current Portion Long-Term Debt
current_portion_longterm_debt = soup.find('td', text="Current Portion Long-Term Debt")
value = current_portion_longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Empréstimos e Financiamentos:', value_formatted)

Dívida de Longo Prazo

In [ ]:
# Encontrando o valor de Long-Term Debt
longterm_debt = soup.find('td', text="Long-Term Debt")
value = longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Dívida de Longo Prazo:', value_formatted)

Capital de Terceiros (Po)

In [ ]:
# Encontrando o valor de Current Portion Long-Term Debt
current_portion_longterm_debt = soup.find('td', text="Current Portion Long-Term Debt")
value = current_portion_longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
current_portion_longterm_debt_in_millions = float(value.replace(',', '')) * 1000000

# Encontrando o valor de Long-Term Debt
longterm_debt = soup.find('td', text="Long-Term Debt")
value = longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
longterm_debt_in_millions = float(value.replace(',', '')) * 1000000

# Somando os valores de Current Portion Long-Term Debt e Long-Term Debt
total_debt = current_portion_longterm_debt_in_millions + longterm_debt_in_millions

# Formatando o valor de volta para uma string com a notação de dólares
if total_debt == 0:
  total_debt_formatted = '$ 0.00'
else:
  total_debt_formatted = '${:,.2f}'.format(total_debt)

# Imprimindo o valor formatado
print('Capital de Terceiros (PO):', total_debt_formatted)


Capital Investido (Ativos Totais)

In [ ]:
# Encontrando o valor de Total Assets
total_assets = soup.find('td', text="Total Assets")
value = total_assets.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Capital Investido (Ativos Totais):', value_formatted)

Caixa e Aplicações Financeiras

In [ ]:
# Encontrando o valor de Cash & Equivalents
cash_equivalents = soup.find('td', text="Cash & Equivalents")
value = cash_equivalents.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Caixa e Aplicações Financeiras:', value_formatted)

Ativo Circulante

In [ ]:
# Encontrando o valor de Total Current Assets
total_current_assets = soup.find('td', text="Total Current Assets")
value = total_current_assets.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Ativo Circulante:', value_formatted)

Ativo Circulante Operacional

In [ ]:
# Encontrando o valor de Total Current Assets
total_current_assets = soup.find('td', text="Total Current Assets")
value_assets = total_current_assets.find_next('td').text

# Encontrando o valor de Cash & Equivalents
cash_equivalents = soup.find('td', text="Cash & Equivalents")
value_cash = cash_equivalents.find_next('td').text

# Removendo as vírgulas e convertendo os valores para float
value_assets_in_millions = float(value_assets.replace(',', '')) * 1000000
value_cash_in_millions = float(value_cash.replace(',', '')) * 1000000

# Calcular o Ativo Circulante Operacional
value_operational_1 = value_assets_in_millions - value_cash_in_millions

# Formatando o valor de volta para uma string com a notação de dólares
if value_operational_1 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_operational_1)

# Imprimindo o valor formatado
print('Ativo Circulante Operacional:', value_formatted)

Passivo Circulante

In [ ]:
# Encontrando o valor de Total Current Liabilities
total_current_liabilities = soup.find('td', text="Total Current Liabilities")
value = total_current_liabilities.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Passivo Circulante:', value_formatted)

Passivo Circulante Operacional

In [ ]:
# Encontrando o valor de Total Current Liabilities
total_current_liabilities = soup.find('td', text="Total Current Liabilities")
value_liabilities = total_current_liabilities.find_next('td').text

# Encontrando o valor de Current Portion Long-Term Debt
current_portion_longterm_debt = soup.find('td', text="Current Portion Long-Term Debt")
value_debt = current_portion_longterm_debt.find_next('td').text

# Removendo as vírgulas e convertendo os valores para float
value_liabilities_in_millions = float(value_liabilities.replace(',', '')) * 1000000
value_debt_in_millions = float(value_debt.replace(',', '')) * 1000000

# Calcular o Passivo Circulante Operacional
value_operational_2 = value_liabilities_in_millions - value_debt_in_millions

# Formatando o valor de volta para uma string com a notação de dólares
if value_operational_2 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_operational_2)

# Imprimindo o valor formatado
print('Passivo Circulante Operacional:', value_formatted)

## Cash Flow Statements

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/cash-flow-statements'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

Despesa Financeira

In [ ]:
# Encontrando o valor de Issuance (Repayment) of Debt
issuance_repayment_of_debt = soup.find('td', text="Issuance (Repayment) of Debt")
value = issuance_repayment_of_debt.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Despesa Financeira:', value_formatted)

Depreciação e Amortização

In [ ]:
# Encontrando o valor de Depreciação e Amortização
Depreciation_Amortization_Depletion = soup.find('td', text="Depreciation/Amortization & Depletion")
value = Depreciation_Amortization_Depletion.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_7 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_7 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_7)

# Imprimindo o valor formatado
print('Depreciação e Amortização:', value_formatted)

CAPEX

In [ ]:
# Encontrando o valor de CAPEX
Property_Equipment = soup.find('td', text="Property & Equipment")
value = Property_Equipment.find_next('td').text

# Removendo as vírgulas e o símbolo negativo, caso exista
if value[0] == '-':
  value_in_millions_8 = float(value[1:].replace(',', '')) * 1000000
else:
  value_in_millions_8 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_8 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_8)

# Imprimindo o valor formatado
print('CAPEX:', value_formatted)

Aquisição / Alienação de Subsidiárias

In [ ]:
# Encontrando o valor de Aquisição / Alienação de Subsidiárias
Acquisition = soup.find('td', text="Acquisition/ Disposition of Subsidiaries")
value = Acquisition.find_next('td').text

# Removendo as vírgulas e o símbolo negativo, caso exista
if value == 'NA':
  value_in_millions_9 = 0
elif value[0] == '-':
  value_in_millions_9 = float(value[1:].replace(',', '')) * 1000000
else:
  value_in_millions_9 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_9 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_9)

# Imprimindo o valor formatado
print('Aquisição / Alienação de Subsidiárias:', value_formatted)

Investimentos

In [ ]:
# Encontrando o valor de Investimentos
Investments = soup.find('td', text="Investments")
value = Investments.find_next('td').text

# Removendo as vírgulas e o símbolo negativo, caso exista
if value[0] == '-':
  value_in_millions_10 = float(value[1:].replace(',', '')) * 1000000
else:
  value_in_millions_10 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_10 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_10)

# Imprimindo o valor formatado
print('Investimentos:', value_formatted)

Outras atividades de investimento

In [ ]:
# Encontrando o valor de Outras atividades de investimento
Other_Investing_Activities = soup.find('td', text="Other Investing Activities")
value = Investments.find_next('td').text

# Removendo as vírgulas e o símbolo negativo, caso exista
if value[0] == '-':
  Other_Investing_Activities = float(value[1:].replace(',', '')) * 1000000
else:
  Other_Investing_Activities = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if Other_Investing_Activities == 0:
  Other_Investing_Activities_formatted = '$ 0.00'
else:
  Other_Investing_Activities_formatted = '${:,.2f}'.format(Other_Investing_Activities)

# Imprimindo o valor formatado
print('Outras atividades de investimento:', Other_Investing_Activities_formatted)

Fluxo de Caixa das atividades de investimentos

In [ ]:
# Calculando FC das atividades de investimentos
FCAI = value_in_millions_8 + value_in_millions_9 + value_in_millions_10 + Other_Investing_Activities

# Formatando o valor de volta para uma string com a notação de dólares
if FCAI == 0:
  FCAI_formatted = '$ 0.00'
else:
  FCAI_formatted = '${:,.2f}'.format(FCAI)

# Imprimindo o valor formatado
print('FCAI:', FCAI_formatted)

## DRE

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/income-statement'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    print(row)

Receita

In [ ]:
# Encontrando o valor de Sales
Sales = soup.find('td', text="Sales")
value = Sales.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions)

# Imprimindo o valor formatado
print('Receita:', value_formatted)

Custos

In [ ]:
# Encontrando o valor de Cost of Goods
Cost_of_Goods = soup.find('td', text="Cost Of Goods")
value = Cost_of_Goods.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_1 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_1 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_1)

# Imprimindo o valor formatado
print('Custos:', value_formatted)

Resultado Bruto

In [ ]:
# Encontrando o valor de Resultado Bruto
Resultado_Bruto = value_in_millions - value_in_millions_1
Resultado_Bruto 

if Resultado_Bruto == 0:
  Resultado_Bruto_formatted = '$ 0.00'
else:
  Resultado_Bruto_formatted = '${:,.2f}'.format(Resultado_Bruto)

# Imprimindo o valor formatado
print('Resultado Bruto:', Resultado_Bruto_formatted)

**EBIT**

In [ ]:
# Encontrando o valor de Pretax Income
Pretax_Income = soup.find('td', text="Pretax Income")
value = Pretax_Income.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_2 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_2 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_2)

# Imprimindo o valor formatado
print('EBIT:', value_formatted)

**EBITDA**

In [ ]:
# Encontrando o valor de EBITDA
EBITDA = value_in_millions_2 + value_in_millions_7
EBITDA 

if EBITDA == 0:
  EBITDA_formatted = '$ 0.00'
else:
  EBITDA_formatted = '${:,.2f}'.format(EBITDA)

# Imprimindo o valor formatado
print('EBITDA:', EBITDA_formatted)

IR

In [ ]:
# Encontrando o valor de Income Taxes
Income_Taxes = soup.find('td', text="Income Taxes")
value = Income_Taxes.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_3 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_3 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_3)

# Imprimindo o valor formatado
print('IR:', value_formatted)

# *NOPAT Restrito* 

In [ ]:
# Encontrando o valor de NOPAT Restrito
NOPAT_Restrito = value_in_millions_2 - value_in_millions_3
NOPAT_Restrito 

if NOPAT_Restrito == 0:
  NOPAT_Restrito_formatted = '$ 0.00'
else:
  NOPAT_Restrito_formatted = '${:,.2f}'.format(NOPAT_Restrito)

# Imprimindo o valor formatado
print('NOPAT Restrito:', NOPAT_Restrito_formatted)

Depesa Financeira

In [ ]:
# Encontrando o valor de Investment Gains/Losses
Investment_Gains_Losses = soup.find('td', text="Investment Gains/Losses")
value = Investment_Gains_Losses.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_4 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_4 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_4)

# Imprimindo o valor formatado
print('Despesa Financeira:', value_formatted)

Lucro Líquido

In [ ]:
# Encontrando o valor de Net Income (GAAP)
Net_Income_GAAP = soup.find('td', text="Net Income (GAAP)")
value = Net_Income_GAAP.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_5 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_5 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_5)

# Imprimindo o valor formatado
print('Lucro Líquido:', value_formatted)

## **FCL**

NOPAT Restrito

In [ ]:
# Encontrando o valor de NOPAT Restrito
NOPAT_Restrito = value_in_millions_2 - value_in_millions_3
NOPAT_Restrito 

if NOPAT_Restrito == 0:
  NOPAT_Restrito_formatted = '$ 0.00'
else:
  NOPAT_Restrito_formatted = '${:,.2f}'.format(NOPAT_Restrito)

# Imprimindo o valor formatado
print('NOPAT Restrito:', NOPAT_Restrito_formatted)

Depreciação e Amortização

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.zacks.com/stock/quote/'+ symbol +'/cash-flow-statements'
user_agent = {'User-agent': 'Mozilla/5.0'}

# Obtendo a página
page = requests.get(url, headers = user_agent)

# Criando o objeto BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Extraindo os valores da tabela
table_rows = soup.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    
# Encontrando o valor de Depreciação e Amortização
Depreciation_Amortization_Depletion = soup.find('td', text="Depreciation/Amortization & Depletion")
value = Depreciation_Amortization_Depletion.find_next('td').text

# Removendo as vírgulas e convertendo o valor para float
value_in_millions_7 = float(value.replace(',', '')) * 1000000

# Formatando o valor de volta para uma string com a notação de dólares
if value_in_millions_7 == 0:
  value_formatted = '$ 0.00'
else:
  value_formatted = '${:,.2f}'.format(value_in_millions_7)

# Imprimindo o valor formatado
print('Depreciação e Amortização:', value_formatted)

FCO

In [ ]:
# Calculando FCO
FCO = NOPAT_Restrito + value_in_millions_7

# Formatando o valor de volta para uma string com a notação de dólares
if FCO == 0:
  FCO_formatted = '$ 0.00'
else:
  FCO_formatted = '${:,.2f}'.format(FCO)

# Imprimindo o valor formatado
print('FCO:', FCO_formatted)

Δ NC Giro

In [ ]:
# Calculando NC Giro
NC_Giro = value_operational_1 - value_operational_2

# Formatando o valor de volta para uma string com a notação de dólares
if NC_Giro == 0:
  NC_Giro_formatted = '$ 0.00'
else:
  NC_Giro_formatted = '${:,.2f}'.format(NC_Giro)

# Imprimindo o valor formatado
print('Δ NC Giro:', NC_Giro_formatted)

CAPEX

In [ ]:
CAPEX_formatted = '${:,.2f}'.format(value_in_millions_8)
print('CAPEX:', CAPEX_formatted)

FCFF

In [ ]:
# Calculando FCFF
if NC_Giro > 0:
  FCFF = FCO - NC_Giro - value_in_millions_8
elif NC_Giro < 0:
  FCFF = FCO + NC_Giro - value_in_millions_8

# Formatando o valor de volta para uma string com a notação de dólares
if FCFF == 0:
  FCFF_formatted = '$ 0.00'
else:
  FCFF_formatted = '${:,.2f}'.format(FCFF)

# Imprimindo o valor formatado
print('FCFF:', FCFF_formatted)

Novas Dívidas

In [ ]:
# Calculando Novas Dívidas
Novas_dividas = value_in_millions_9 + value_in_millions_10

# Formatando o valor de volta para uma string com a notação de dólares
if Novas_dividas == 0:
  Novas_dividas_formatted = '$ 0.00'
else:
  Novas_dividas_formatted = '${:,.2f}'.format(Novas_dividas)

# Imprimindo o valor formatado
print('Novas Dívidas:', Novas_dividas_formatted)

IR

In [ ]:
# Imprimindo o valor formatado
IR_formatted = '${:,.2f}'.format(value_in_millions_3)
print('IR:', IR_formatted)

**FCDA**

In [ ]:
# Calculando FCDA
FCDA = FCFF + Novas_dividas + value_in_millions_3

# Formatando o valor de volta para uma string com a notação de dólares
if FCDA == 0:
  FCDA_formatted = '$ 0.00'
else:
  FCDA_formatted = '${:,.2f}'.format(FCDA)

# Imprimindo o valor formatado
print('FCDA:', FCDA_formatted)

# **CAPM - Custo do Capital Próprio**

Custo de Capital Próprio

In [ ]:
# Calculando Ke

Ke = interest_rate + (beta*((mean_yearly_return) - interest_rate))
Ke_formatted = '{:,.2f}%'.format(Ke)
print ('Ke:' ,Ke_formatted)

Ki (Kd) - Custo da Dívida

In [ ]:
# Calculando Ki (Kd) - Custo da Dívida
if value_in_millions_4 == 0:
  Ki = Rf + inflation
else:
  Ki = (value_in_millions_4 / total_debt) + 1

# Formatando o valor de volta para uma string com a notação de dólares
if Ki == 0:
  Ki_formatted = ' 0.00 %'
else:
  Ki_formatted = '{:,.2f} %'.format(Ki)

# Imprimindo o valor formatado
print('Ki (Kd) - Custo da Dívida:', Ki_formatted)

Po e PL

In [ ]:
Po = total_debt
PL = value_in_millions_12
Po_formatted = '${:,.2f}'.format(total_debt)
PL_formatted = '${:,.2f}'.format(value_in_millions_12)
print ('PL:' , PL_formatted)
print ('Po:' , Po_formatted)

PL / (Po+PL)

In [ ]:
# Calculando PL / (Po+PL)
if value_in_millions_12 == 0:
  PL_Po_PL = 0.0
else:
  PL_Po_PL = value_in_millions_12 / (value_in_millions_12 + total_debt)

# Formatando o valor de volta para uma string com a notação de dólares
if PL_Po_PL == 0:
   PL_Po_PL_formatted = ' 0.0'
else:
  PL_Po_PL_formatted = '{:,.2f}'.format(PL_Po_PL)

# Imprimindo o valor formatado
print('PL / (Po+PL):', PL_Po_PL_formatted)

Po / (Po+PL)

In [ ]:
# Calculando Po / (Po+PL)
if total_debt == 0:
  Po_Po_PL = 0.0
else:
  Po_Po_PL = total_debt / (value_in_millions_12 + total_debt)

# Formatando o valor de volta para uma string com a notação de dólares
if Po_Po_PL == 0:
   Po_Po_PL_formatted = ' 0.0'
else:
  Po_Po_PL_formatted = '{:,.2f}'.format(Po_Po_PL)

# Imprimindo o valor formatado
print('Po / (Po+PL):', Po_Po_PL_formatted)

**WACC**

In [ ]:
# Calculando WACC
wacc = (Ke * PL_Po_PL ) + (Ki * Po_Po_PL)
wacc_formatted = '{:,.2f}%'.format(wacc)
print ('WACC:' , wacc_formatted)

**Custo de Oportunidade**

In [ ]:
# Calculando Co = PL/Ke
Co = value_in_millions_12*(Ke/100)
Co_formatted = '${:,.2f}'.format(Co)
value_in_millions_12_formated = '${:,.2f}'.format(value_in_millions_12)
print('PL:' ,value_in_millions_12_formated)
print ('Ke:' ,Ke_formatted)
print ('Co:' , Co_formatted)


**Lucro Econômico (EVA)**

In [ ]:
# Calculando LE = LL - Co
LE = value_in_millions_5 - Co
LE_formatted = '${:,.2f}'.format(LE)
print ('LE:' , LE_formatted)


ROE > KE

In [ ]:
# Calcular a diferença entre o ROE e o ke
ROE_KE = roe - Ke

print('ROE: {:.2f}%'.format(roe))
print('KE: {:.2f}%'.format(Ke))
print('Diferença de ROE x Ke: {:.2f}%'.format(ROE_KE))

if ROE_KE > 0:
    print("SIM, ROE é maior que KE")
elif ROE_KE < 0:
    print("NÃO, ROE é menor que KE")
else:
    print("IGUAL, ROE é igual a KE")

ROI > WACC

In [ ]:
# Calcular a diferença entre o ROI e o WACC
ROI_WACC = roi - wacc

print('ROI: {:.2f}%'.format(roi))
print('WACC: {:.2f}%'.format(wacc))
print('Diferença de ROI x WACC: {:.2f}%'.format(ROI_WACC))

if ROI_WACC > 0:
    print("SIM, ROI é maior que WACC")
elif ROI_WACC < 0:
    print("NÃO, ROI é menor que WACC")
else:
    print("IGUAL, ROI é igual a WACC")

# **Valuation**

***Nº de Ações***

In [ ]:
# Localizando o número de ações
shs_outstand_str = fundamentals.loc['Shs Outstand']['Values']
shs_outstand_str = shs_outstand_str.strip()

if shs_outstand_str[-1] == 'B':
    shs_outstand = float(shs_outstand_str[:-1]) * 1000000000
elif shs_outstand_str[-1] == 'M':
    shs_outstand = float(shs_outstand_str[:-1]) * 1000000
else:
    shs_outstand = float(shs_outstand_str)

shs_outstand_formatted = '{:,d}'.format(int(shs_outstand))

print(f'Nº de Ações: {shs_outstand_formatted}')

**Crescimento dos Lucros líquidos ponderado ao Mercado - gLL + Média Mercado**

In [ ]:
#Crescimento dos Lucros líquidos ponderado ao Mercado - gLL + Média Mercado
# O correto seria somar ponderar o gll + a média do crescimento do ROE de 5Y)
gll_M = (gll + (mean_yearly_return/100))/2
print(gll_M)

**bNOPAT**

In [ ]:
# Calculando bNOPAT
#bNOPAT = (CAPEX - Amortização e Depeciação) / EBIT
bNOPAT = (value_in_millions_8 - value_in_millions_7)/value_in_millions_2

print(bNOPAT)

**gNOPAT**

In [ ]:
# Calculando gNOPAT
# gNOPAT = bNOPAT * ROI
gNOPAT = bNOPAT * roi_perc
print (gNOPAT)

**Crescimeto na perpetuidade**

In [ ]:
# Calculando Crescimeto na perpetuidade
CP = 2/100
CP

**FCFF**

In [ ]:
FCFF
# Imprimindo o valor formatado
print('FCFF:', FCFF_formatted)

**VP**

In [ ]:
# A Taxa de desconto é calculada usando a formula (1+wacc)^n
# Ano 0
Ano_0 = 0
EBITDA_0 = 0.0
FCDE_0 = FCFF
Taxa_Desconto_0 = 0.0
VP_0 = FCDE_0

# Ano 1
Ano_1 = 1
EBITDA_1 = FCDE_0 * gll_M
FCDE_1 = FCDE_0 + EBITDA_1
Taxa_Desconto_1 = pow (1 + wacc/100, Ano_1)
VP_1 = FCDE_1/Taxa_Desconto_1

# Ano 2
Ano_2 = 2
EBITDA_2 = FCDE_1 * gll_M
FCDE_2 = FCDE_1 + EBITDA_2
Taxa_Desconto_2 = pow (1 + wacc/100, Ano_2)
VP_2 = FCDE_2/Taxa_Desconto_2

# Ano 3
Ano_3 = 3
EBITDA_3 = FCDE_2 * gll_M
FCDE_3 = FCDE_2 + EBITDA_3
Taxa_Desconto_3 = pow (1 + wacc/100, Ano_3)
VP_3 = FCDE_3/Taxa_Desconto_3

# Ano 4
Ano_4 = 4
EBITDA_4 = FCDE_3 * gll_M
FCDE_4 = FCDE_3 + EBITDA_4
Taxa_Desconto_4 = pow (1 + wacc/100, Ano_4)
VP_4 = FCDE_4/Taxa_Desconto_4

# Ano 5
Ano_5 = 5
EBITDA_5 = FCDE_4 * gll_M
FCDE_5 = FCDE_4 + EBITDA_5
Taxa_Desconto_5 = pow (1 + wacc/100, Ano_5)
VP_5 = FCDE_5/Taxa_Desconto_5

# Ano 6
Ano_6 = 6
EBITDA_6 = FCDE_5 * gll_M
FCDE_6 = FCDE_5 + EBITDA_6
Taxa_Desconto_6 = pow (1 + wacc/100, Ano_6)
VP_6 = FCDE_6/Taxa_Desconto_6

# Ano 7
Ano_7 = 7
EBITDA_7 = FCDE_6 * gll_M
FCDE_7 = FCDE_6 + EBITDA_7
Taxa_Desconto_7 = pow (1 + wacc/100, Ano_7)
VP_7 = FCDE_7/Taxa_Desconto_7

# Ano 8
Ano_8 = 8
EBITDA_8 = FCDE_7 * gll_M
FCDE_8 = FCDE_7 + EBITDA_8
Taxa_Desconto_8 = pow (1 + wacc/100, Ano_8)
VP_8 = FCDE_8/Taxa_Desconto_8

# Ano 9 
Ano_9 = 9
EBITDA_9 = FCDE_8 * gll_M
FCDE_9 = FCDE_8 + EBITDA_9
Taxa_Desconto_9 = pow (1 + wacc/100, Ano_9)
VP_9 = FCDE_9/Taxa_Desconto_9

# Ano 10 
Ano_10 = 10
EBITDA_10 = FCDE_9 * gll_M
FCDE_10 = FCDE_9 + EBITDA_10
Taxa_Desconto_10 = pow (1 + wacc/100, Ano_10)
VP_10 = FCDE_10/Taxa_Desconto_10

# Ano 11 - Perpetuidade
Ano_11 = 11
FCDE_11 = FCDE_10 
Taxa_Desconto_11 =  (wacc/100 - (gNOPAT))*100 
VP_perpeturidade = (FCDE_11/(Taxa_Desconto_11)*100)
VP_11 = VP_perpeturidade / (Taxa_Desconto_10)


print("Ano  |     EBITDA      |     FCDE          |   Taxa de Desconto     |        VP")
print("{:^4} | ${:^11,.2f}    | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_0, EBITDA_0, FCDE_0, Taxa_Desconto_0, VP_0))
print("{:^4} | ${:^11,.2f}  | ${:^11,.2f}    | {:^15.2f}        | ${:^11,.2f}".format(Ano_1, EBITDA_1, FCDE_1, Taxa_Desconto_1, VP_1))
print("{:^4} | ${:^11,.2f}  | ${:^11,.2f}   | {:^15.2f}        | ${:^11,.2f}".format(Ano_2, EBITDA_2, FCDE_2, Taxa_Desconto_2, VP_2))
print("{:^4} | ${:^11,.2f}  | ${:^11,.2f}   | {:^15.2f}        | ${:^11,.2f}".format(Ano_3, EBITDA_3, FCDE_3, Taxa_Desconto_3, VP_3))
print("{:^4} | ${:^11,.2f}  | ${:^11,.2f}   | {:^15.2f}        | ${:^11,.2f}".format(Ano_4, EBITDA_4, FCDE_4, Taxa_Desconto_4, VP_4))
print("{:^4} | ${:^11,.2f} | ${:^11,.2f}   | {:^15.2f}        | ${:^11,.2f}".format(Ano_5, EBITDA_5, FCDE_5, Taxa_Desconto_5, VP_5))
print("{:^4} | ${:^11,.2f} | ${:^11,.2f}   | {:^15.2f}        | ${:^11,.2f}".format(Ano_6, EBITDA_6, FCDE_6, Taxa_Desconto_6, VP_6))
print("{:^4} | ${:^11,.2f} | ${:^11,.2f}   | {:^15.2f}        | ${:^11,.2f}".format(Ano_7, EBITDA_7, FCDE_7, Taxa_Desconto_7, VP_7))
print("{:^4} | ${:^11,.2f} | ${:^11,.2f} | {:^15.2f}        | ${:^11,.2f}".format(Ano_8, EBITDA_8, FCDE_8, Taxa_Desconto_8, VP_8))
print("{:^4} | ${:^11,.2f} | ${:^11,.2f} | {:^15.2f}        | ${:^11,.2f}".format(Ano_9, EBITDA_9, FCDE_9, Taxa_Desconto_9, VP_9))
print("{:^4} | ${:^11,.2f} | ${:^11,.2f} | {:^15.2f}        | ${:^11,.2f}".format(Ano_10, EBITDA_10, FCDE_10, Taxa_Desconto_10, VP_10))
print("{:^4}                   | ${:^11,.2f} | {:^15.2f}        | ${:^11,.2f}".format(Ano_11, FCDE_11, Taxa_Desconto_11, VP_11))

**VP**

In [ ]:
#Encontrando o Valor Patrimonial (VP)
VP = VP_0 + VP_1 + VP_2 + VP_3 + VP_4 + VP_5 + VP_6 + VP_7 + VP_8 + VP_9 + VP_10 + VP_11
VP_formatted = '${:,.2f}'.format(VP)
print ('VP:' , VP_formatted)

# **Preço Justo da Ação**

In [ ]:
#Encontrando o Preço Justo da Ação
PJA = VP / shs_outstand
PJA_formatted = '${:,.2f}'.format(PJA)
print ('PJA:' , PJA_formatted)

# Calcular o preço atual
price_str = fundamentals.loc['Price']['Values']
price = float(price_str)
print(f'Preço Atual: ${price_str}')

Margem_P = PJA - price
Margem_P_formatted = '${:,.2f}'.format(Margem_P)
print('Margem:' , Margem_P_formatted)

Margem_Perc = (Margem_P / PJA)*100
Margem_Perc_formatted = '{:,.2f}%'.format(Margem_Perc)
print('Margem Percentual:' , Margem_Perc_formatted)



# **AVALIAÇÃO PELO LUCRO EM EXCESSO**

In [ ]:
# Calculando Vo = Capital Investido + PV do Retorno em Excesso
#PV do Retorno em Excesso = Goodwill

Goodwill = LE / Ke
Goodwill_formatted = '${:,.2f}'.format(Goodwill)
print('Goodwill',Goodwill_formatted)

Vo_excesso = PL + Goodwill
Vo_excesso_formatted = '${:,.2f}'.format(Vo_excesso)
print('Vo',Vo_excesso_formatted)

#Encontrando o Preço Justo da Ação pelo Lucro em Excesso
PJA_E = Vo_excesso / shs_outstand
PJA_E_formatted = '${:,.2f}'.format(PJA_E)
print ('PJA pelo Lucro em Excesso:' , PJA_E_formatted)

# Calcular o preço atual
price_str = fundamentals.loc['Price']['Values']
price = float(price_str)
print(f'Preço Atual: ${price_str}')


# **Valor Intrínseco**

In [ ]:
import math

eps = float(eps)
VPA = float(VPA)

# Calcular o valor intrínseco em dólar
valor_intrinseco = math.sqrt(eps * VPA * 22.5)

# Calcular o preço atual

price_str = fundamentals.loc['Price']['Values']
price = float(price_str)
print(f'Preço Atual: ${price_str}')

# Calcular a diferença entre o valor intrínseco e o preço atual
diferenca = valor_intrinseco - price

# Calcular a diferença percentual
dif_percentual = diferenca / valor_intrinseco * 100

# Verificar se está com ágio ou deságio
if price > valor_intrinseco:
    agio_desagio = 'ágio'
else:
    agio_desagio = 'deságio'

# Exibir os resultados
print(f"Valor intrínseco: ${valor_intrinseco:.2f}")
print(f"Diferença: ${diferenca:.2f}")
print(f"Margem percentual: {dif_percentual:.2f}% {agio_desagio}")

if agio_desagio == 'ágio':
    recomendacao = 'Recomendação: Venda'
else:
    recomendacao = 'Recomendação: Compra'

print(recomendacao)

# **Comparativo entre Avaliações**

In [ ]:
print(f'Preço Atual: ${price_str}')
print(f"Valor intrínseco: ${valor_intrinseco:.2f}")
print ('Preço Valuation:' , PJA_formatted)
print ('PJA pelo Lucro em Excesso:' , PJA_E_formatted)

# **Retorno S&P 500 - 10 Y x Ticker em análise**

**Retorno S&P 500**

In [ ]:
Ten_year_return = sp500['acum_return'].iloc[-1] * 100 - 100
print(f'Retorno acumulado 10Y: {Ten_year_return:.1f}%')

mean_yearly_return = sp500['daily_return'].mean() * 100 * 365
print(f'Retorno médio anual 10Y: {mean_yearly_return:.2f}%')

**Retorno Ticker**

In [ ]:
def calcula_retorno(dataframe):
  df = dataframe.copy()  
  df['daily_return']        = df.iloc[:,0].pct_change()
  df['acum_return']         = (1 +  df['daily_return']).cumprod()
  df['acum_return'].iloc[0] = 1
  df.rename(columns={df.columns[0]: "price" }, inplace=True)
  return df

ticker = yf.download(symbol)[['Adj Close']]
ticker

ticker = calcula_retorno(ticker)
ticker

Ten_year_return_ticker = ticker['acum_return'].iloc[-1] * 100 - 100
print(f'Retorno acumulado 10Y do Ticker: {Ten_year_return_ticker:.1f}%')

mean_yearly_return_ticker = ticker['daily_return'].mean() * 100 * 365
print(f'Retorno médio anual 10Y do Ticker: {mean_yearly_return_ticker:.2f}%')


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sp500.index[-252*10:], y=sp500['price'][-252*10:]))
fig.update_layout(yaxis_type="log")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=ticker.index[-252*10:], y=ticker['price'][-252*10:]))
fig.update_layout(yaxis_type="log")
fig.show()

**Análise correlacionadas ao Benchmarket (S&P 500)**

In [ ]:
tickers = [symbol,"^GSPC"]
dados_yahoo = web.get_data_yahoo(tickers, period="10y")["Adj Close"]

In [ ]:
dados_yahoo.columns = [symbol,"SP500"]
dados_yahoo

RETORNO DIARIOS

In [ ]:
retorno = dados_yahoo.pct_change()
retorno

RETORNO ACUMULADO

In [ ]:
retorno_acumulado = (1 + retorno).cumprod()
retorno_acumulado.iloc[0] = 1
retorno_acumulado

In [ ]:
carteira = 10000 * retorno_acumulado.iloc[:, :5]
carteira["saldo"] = carteira.sum(axis=1)
carteira["retorno"] = carteira["saldo"].pct_change()
carteira

# **RESULTADOS**

In [ ]:
pf.create_full_tear_sheet(carteira["retorno"], benchmark_rets=retorno["SP500"])

In [ ]:
fig, ax1 = plt.subplots(figsize=(22,12))
pf.plot_rolling_beta(carteira["retorno"],factor_returns=retorno["SP500"], ax=ax1)
plt.ylim((0.5,2))